## 0.0 Planejamento da solução 

**Input - Entrada** 

1. Problema de Negócio
    -- Selecionar os clientes mais valiosos para integrar um programa de fidelização
    

2. Conjunto de dados
    -- Vendas de um e-ccomerce online durante um período de 1 ano.   
    


**Output - Saída**

A indicação de pessoas para fazer parte do programa de fidelidade "INSIDERS".

Relatório com as respostas para as seguintes perguntas:
    Quem são as pessoas elegíveis para participar do programa de Insiders ?
    
    1. Quantos clientes farão parte do grupo?
    
    2. Quais as principais características desses clientes ?
    
    3. Qual a porcentagem de contribuição do faturamento, vinda do Insiders ?
    
    4. Qual a expectativa de faturamento desse grupo para os próximos meses ?
    
    5. Quais as condições para uma pessoa ser elegível ao Insiders ?
    
    6. Quais as condições para uma pessoa ser removida do Insiders ?
    
    7. Qual a garantia que o programa Insiders é melhor que o restante da base ?
    
    8. Quais ações o time de marketing pode realizar para aumentar o faturamento?

 **Tasks - Processo**

1. Quem são as pessoas elegíveis para participar do programa de Insiders ?
    --O que é ser elegível. Para a empresa o que são clientes com maior valor?

        Faturamento:
            Alto Ticket MédioAlto LTVBaixa Recência ou Alta Frequência ( tempo entre as compras )

            Alto Basket Size ( quantidade média de produtos comprados )

            Baixa probabilidade de Churn
            
            Previsão alta de LTV
            
            Alta propensão de compra

        Custo:

            Baixo número de devoluções

        Experiência de compra:  

            Média alta de avaliações
            
2. Quantos clientes farão parte de cada grupo
     --Número total de clientes
         %do grupo de insiders
     
3. Quais as principais características desses clientes?
      -- Escrever caracteristicas desses clientes
              idade
              Localização
       
      -- Escrever caracteristicas do consumo
              Atributos de clusterização
              
4. Qual a porcentagem de contribuição do faturamento, vinda do Insiders ?
        --Faturamento total do ano
        --Faturamento do grupo de insiders        
        

5. Qual a expectativa de faturamento desse grupo para os próximos meses ?
        --LTV do grupo de insiders
        --Analise de cohort
   
6. Quais as condições para uma pessoa ser elegível ao Insiders ?
        -- Definir a periodicidade
        -- Pessoas com caracteristicas parecidas com os outros insders do grupo


7. Quais as condições para uma pessoa ser removida do Insiders ?
         -- Definir a periodicidade
         -- Não estar dentro das características acima

8. ual a garantia que o programa Insiders é melhor que o restante da base ?
        -- teste A/B
        --teste A/B baysiano
        -- Teste de hipoteses
        
        
9. Quais ações o time de marketing pode realizar para aumentar o faturamento? 
        -- Desconto
        -- frete
        -- Visita a empresa
        -- brinde
    
        



## 0.0 Imports

In [40]:
import re
import pandas    as pd
import numpy     as np
import seaborn   as sns

import umap.umap_   as umap

from matplotlib          import pyplot        as plt
from sklearn             import cluster       as c
from plotly              import express       as px
from sklearn             import ensemble      as en
from sklearn             import metrics       as m
from sklearn             import preprocessing as pp
from sklearn             import decomposition as dd
from sklearn             import mixture       as mx
from sklearn.manifold    import TSNE

## 0.2 Load Dataset

In [41]:
df_raw = pd.read_csv('Data/ecommerce.csv',encoding='cp1252')
df_raw.drop('Unnamed: 8', axis=1, inplace=True)

# 1.0 Data Descriptions

In [42]:
df1 = df_raw.copy()

In [43]:
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.0,United Kingdom


## 1.1 Rename Columns

In [44]:
df1.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [45]:
cols_new =['Invoice_No', 'Stock_Code', 'Description', 'Quantity', 'Invoice_Date',
       'Unit_Price', 'Customer_ID', 'Country']


cols_new = list( (map(lambda x: x.lower(), cols_new)) )


df1.columns = cols_new

## 1.2 Data Dimensions

In [46]:
print(f'Numbers of rows: { df1.shape[0] }\n')
print(f'Numbers of columns: { df1.shape[1] }')

Numbers of rows: 541909

Numbers of columns: 8


## 1.3 Data Types

In [47]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

## 1.4 Check NA

In [48]:
df1.isna().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

## 1.5 Replace NA

In [49]:
# TThe strategy is replace NAs to do a general study
df_missing =df1.loc[df1['customer_id'].isna(), :]
df_not_missing = df1.loc[~df1['customer_id'].isna(), :]


In [50]:
# Create reference

df_backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates())
df_backup['customer_id'] = np.arange( 19000, 19000+len(df_backup), 1)

#Merge original with refrece dataframe
df1 = pd.merge(df1, df_backup, on='invoice_no', how='left')

#coalesce
df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])

#Drop extra columns
df1 = df1.drop( columns=['customer_id_x', 'customer_id_y'], axis=1)
df1.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,United Kingdom,17850.0
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,United Kingdom,17850.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,United Kingdom,17850.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,United Kingdom,17850.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,United Kingdom,17850.0


In [51]:
df1.isna().sum()

invoice_no         0
stock_code         0
description     1454
quantity           0
invoice_date       0
unit_price         0
country            0
customer_id        0
dtype: int64

## 1.6 Change dtypes

In [52]:
# invoicedate
df1['invoice_date'] = pd.to_datetime( df1['invoice_date'], format='%d-%b-%y')

#customerid
df1['customer_id'] = df1['customer_id'].astype('int64')


In [53]:
df1.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
country                 object
customer_id              int64
dtype: object

## 1.7 Descriptive Statistics

In [54]:
num_attributes = df1.select_dtypes( include=['int64', 'float64'])
cat_attributes = df1.select_dtypes( exclude=['int64', 'float64', 'datetime64[ns]'])

### 1.7.1 Numerical Attributes

In [55]:
#central tendency - mean, median, 
ct1 = pd.DataFrame(num_attributes.apply(np.mean) ).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

#Dispersion - std, 
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(np.min)).T
d3 = pd.DataFrame(num_attributes.apply(np.max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew()) ).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis()) ).T

#Merge
n = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
n.columns = ['attributes', 'min', 'max', 'range','mean', 'median', 'std','skew','kurtosis']
n

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,quantity,-80995.00,80995.0,161990.00,9.552250,3.00,218.080957,-0.264076,119769.160031
1,unit_price,-11062.06,38970.0,50032.06,4.611114,2.08,96.759764,186.506972,59005.719097
2,customer_id,12346.00,22709.0,10363.00,16688.840453,16249.00,2911.408666,0.487449,-0.804287


### 1.7.2 Categorical Attributes

### Invoice_no

In [56]:
# problema: O invoice tem numeros e letras

df_letter_invoices = df1.loc[df1['invoice_no'].apply( lambda x: bool( re.search( '[^0-9]+', x ) ) ), :]

#Verificar se todos os invoices são negativos
print(f'total de invoices com letras: {len(df_letter_invoices)}')

#Verificar se todos os invoices são negativos
print(f'total de invoices com valores negativos: {len(df_letter_invoices[ df_letter_invoices["quantity"] < 0 ])}')


total de invoices com letras: 9291
total de invoices com valores negativos: 9288


Todos os invoices constituidos por numeros e letras são negativos, então vou assumir que é um retorno.

## Stoke_code

In [57]:
#at_least_one_string = len(cat_attributes.loc[ cat_attributes['stock_code'].apply( lambda x: bool( re.search('[^0-9]+', x))), 'stock_code'].drop_duplicates())

cat_attributes.loc[ cat_attributes['stock_code'].apply( lambda x:  bool( re.search( '^[a-zA-z]+$', x))),'stock_code'].unique()

#Ação
# Remover 'POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK'

array(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY',
       'DCGSSGIRL', 'PADS', 'B', 'CRUK'], dtype=object)

### Country

In [58]:
len(df1['country'].unique())

38

In [59]:
#Representatividade de cada país nas compras

df1['country'].value_counts(normalize=True)

United Kingdom          0.914320
Germany                 0.017521
France                  0.015790
EIRE                    0.015124
Spain                   0.004674
Netherlands             0.004375
Belgium                 0.003818
Switzerland             0.003694
Portugal                0.002803
Australia               0.002323
Norway                  0.002004
Italy                   0.001482
Channel Islands         0.001399
Finland                 0.001283
Cyprus                  0.001148
Sweden                  0.000853
Unspecified             0.000823
Austria                 0.000740
Denmark                 0.000718
Japan                   0.000661
Poland                  0.000629
Israel                  0.000548
USA                     0.000537
Hong Kong               0.000531
Singapore               0.000423
Iceland                 0.000336
Canada                  0.000279
Greece                  0.000269
Malta                   0.000234
United Arab Emirates    0.000125
European C

In [60]:
df1[['customer_id', 'country']].drop_duplicates().groupby('country').count().reset_index().sort_values('customer_id', ascending=False).head()

,country,customer_id
36,United Kingdom,7587
14,Germany,95
13,France,90
10,EIRE,44
31,Spain,31


#  2.0 Filtering Variable

In [61]:
df2 =df1.copy()

In [62]:
df2.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
country                 object
customer_id              int64
dtype: object

In [63]:
#=====================Numerical Attributes=================================
#Quantity
df2 = df2.loc[df2['unit_price']>= 0.04]


#=====================Caterical Attributes=================================
df2 = df2[~df2['stock_code'].isin( ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY',
       'DCGSSGIRL', 'PADS', 'B', 'CRUK'])]

#Description
df2 = df2.drop(columns='description', axis=1)

#map
df2 = df2[~df2['country'].isin(['European Community', 'Unspecified'])]

#Quantity
df_return = df2.loc[df1['quantity']<0, :]
df2_purchase = df2.loc[df1['quantity']>= 0, :]

#Bad Users
df2 = df2[~df2['customer_id'].isin( [16446] )]



#  3.0 Feature Engeneering

In [64]:
df3= df2.copy()

## 3.1 Feature Creation

In [65]:
# Data reference
df_ref = df3.drop(['invoice_no', 'stock_code', 'quantity', 'invoice_date', 'unit_price', 
                   'country'], axis=1).drop_duplicates(ignore_index=True)

In [66]:
df_ref.shape

(5786, 1)

### 3.1.1 Gross Revenue

In [67]:
#Gross Revenue (Faturamento) - Quantidade * preco
df2_purchase[ 'gross_revenue'] = df2_purchase['quantity'] * df2_purchase['unit_price']

#Monetary
df_monetary = df2_purchase[['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()

df_ref = pd.merge(df_ref, df_monetary, on='customer_id', how='left')


### 3.1.2 Recency - Day from last purchase

In [68]:
# Recency - Last day purchase
df_recency = df2_purchase.loc[:, ['customer_id', 'invoice_date']].groupby( 'customer_id' ).max().reset_index()
df_recency['recency_days'] = ( df2['invoice_date'].max() - df_recency['invoice_date'] ).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy()
df_ref = pd.merge( df_ref, df_recency, on='customer_id', how='left' )
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
dtype: int64

### 3.1.5 Quantity of products purchased

In [69]:
#Products number
df_freq = (df2_purchase.loc[:,['customer_id', 'stock_code']].groupby('customer_id')
                                                            .count()
                                                            .reset_index()
                                                            .rename(columns={'stock_code':'qtde_products'}))
           
df_ref =pd.merge(df_ref, df_freq, on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qtde_products    91
dtype: int64

In [70]:
df_ref.head()

,customer_id,gross_revenue,recency_days,qtde_products
0,17850,5391.21,372.0,297.0
1,13047,3232.59,56.0,171.0
2,12583,6705.38,2.0,232.0
3,13748,948.25,95.0,28.0
4,15100,876.00,333.0,3.0


### 3.1.8 Frequency Purchase

In [71]:
#COmo Fazer todo esse processo apenas uma vez
df_aux = (df2_purchase[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates()
                                                           .groupby('customer_id')
                                                           .agg(max_ =('invoice_date', 'max'),
                                                                min_ =('invoice_date', 'min'),
                                                                days_ =('invoice_date', lambda x:(( x.max() - x.min()).days)+1),
                                                                buy_ =('invoice_date', 'count'))).reset_index()
#Frequency
df_aux['frequency'] =df_aux[['buy_', 'days_']].apply( lambda x: x['buy_']/ x['days_'] if x['days_'] != 0 else 0, axis=1)

#Merge
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'frequency']], how='left', on='customer_id')

In [72]:
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qtde_products    91
frequency        91
dtype: int64

### 3.1.9 Number of Returns

In [73]:
df_return = df_return[['customer_id', 'quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity':'qtde_returns'})
df_return['qtde_returns'] = df_return['qtde_returns']* -1


#Merge
df_ref = pd.merge(df_ref, df_return, on='customer_id', how='left')
df_ref.loc[df_ref['qtde_returns'].isna(), 'qtde_returns'] =0 
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qtde_products    91
frequency        91
qtde_returns      0
dtype: int64

#  4.0 EDA - Exploratory Data Analysis

In [74]:
df4 =df_ref.dropna()

In [75]:
df4.isna().sum()

customer_id      0
gross_revenue    0
recency_days     0
qtde_products    0
frequency        0
qtde_returns     0
dtype: int64

## 4.3 Estudo do Espaço

In [76]:
cols_selected = ['customer_id', 'gross_revenue','recency_days', 'qtde_products', 'frequency', 'qtde_returns']
df43 = df4[cols_selected].copy()

In [77]:
mm = pp.MinMaxScaler()

df43['gross_revenue']         =  mm.fit_transform(df43[['gross_revenue']])
df43['recency_days']          =  mm.fit_transform(df43[['recency_days']])
df43['qtde_products']         =  mm.fit_transform(df43[['qtde_products']])
df43['frequency']             =  mm.fit_transform(df43[['frequency']])
df43['qtde_returns']          =  mm.fit_transform(df43[['qtde_returns']])

### 4.3.4 Tree badesd embeding

In [78]:
#training dataset
X= df43.drop(columns=['customer_id', 'gross_revenue'])
 
y = df43['gross_revenue']

#model definition
rf_model = en.RandomForestRegressor(n_estimators=100, random_state=42)

# Model Training
rf_model.fit(X, y)
#Leaf

#Dataframe
df_leaf = pd.DataFrame(rf_model.apply(X))

In [79]:
# Reduzer dimensionality
reducer = umap.UMAP( random_state=42 )
embedding = reducer.fit_transform( df_leaf )

# embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:, 0]
df_tree['embedding_y'] = embedding[:, 1]

C:\Users\PC\anaconda3\envs\pa005\lib\site-packages\sklearn\manifold\_spectral_embedding.py:259: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


# 7.0 Hyper Parameter Fine-Tuning

In [80]:
X=df_tree.copy()

In [81]:
X.head()

,embedding_x,embedding_y
0,9.374369,11.323303
1,14.677434,13.025318
2,9.889122,11.613262
3,-5.138314,-1.075224
4,18.822596,-4.035024


# 8.0 Model Training

## 8.1 Final Model

In [82]:
k=8
#Model definition
gmm_model = mx.GaussianMixture(n_components=k, n_init=300, random_state=42)


#Model training
gmm_model.fit( X )

#Clustering
labels = gmm_model.predict(X)

## 8.2 Cluster Validation

In [83]:
# WSS ( within-cluster sum of square)
#print(f'WSS value: {kmeans.inertia_}')

# SS (Silhouette Score)
print( 'SS value: {}'.format( m.silhouette_score( X, labels, metric='euclidean' ) ) )

SS value: 0.29702234268188477


# 9.0 Cluster Analysis

In [93]:
df92 = df4[ cols_selected ].copy()
df92['cluster'] = labels

#change dtypes
df92['recency_days'] = df92['recency_days'].astype('int64')
df92['qtde_products'] = df92['qtde_products'].astype('int64')
df92['qtde_returns'] = df92['qtde_returns'].astype('int64')

In [94]:
# cluster number
df_cluster = df92[['customer_id', 'cluster']].groupby('cluster').count().reset_index()
df_cluster['perc_customer'] = (df_cluster['customer_id']/ df_cluster['customer_id'].sum()*100)

#Gross revenue

df_avg_gross_revenue = df92[['gross_revenue', 'cluster']].groupby('cluster').mean().reset_index()

df_cluster = pd.merge(df_cluster, df_avg_gross_revenue, how='inner', on='cluster')

#Recency_days
df_avg_recency_days = df92[['recency_days', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_recency_days, how='inner', on='cluster')

#Products
df_qtde_products = df92[['qtde_products', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_qtde_products, how='inner', on='cluster')

#frequency
df_frequency = df92[['frequency', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster =pd.merge(df_cluster, df_frequency, how='inner', on='cluster')

#Returns
df_qtde_returns = df92[['qtde_returns', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster =pd.merge(df_cluster, df_qtde_returns, how='inner', on='cluster')

df_cluster.sort_values('gross_revenue',ascending=False)

,cluster,customer_id,perc_customer,gross_revenue,recency_days,qtde_products,frequency,qtde_returns
0,0,1038,18.226514,5605.191840,77.370906,305.047206,0.342201,149.015414
3,3,311,5.460931,2012.408585,57.006431,102.379421,0.222812,6.440514
4,4,685,12.028095,1665.379708,75.182482,53.982482,0.153353,8.318248
5,5,683,11.992976,1058.016164,103.714495,49.951684,0.372151,9.117130
2,2,1001,17.576822,1022.538811,124.855145,66.663337,0.744456,3.300699
7,7,365,6.409131,567.900685,63.123288,14.358904,0.217756,0.857534
6,6,339,5.952590,481.663658,237.734513,43.106195,0.726372,0.510324
1,1,1273,22.352941,316.630738,170.395915,16.695208,0.991522,4.459544


In [95]:
#Nomear os cluster
#1 -> ouro
#7 -> safira
#2 -> diamante
#3 -> jade
#6 -> prata
#4 -> perola
#5-> esmeralda
#0-> rubi


### Cluster 01:  ( Candidato ao cluster ouro )
 
    - Número de customers: 468 (15,7% do customers )
    
    - Faturamento médio: $ 8836,13
    
    - Recência em média: 21 dias
    
    - Média de produtos comprados:424
    
    - Frequencia média: 0,09 produto/dia 
    
### Cluster 02:
    - Número de customers: 417 (14.04 % do customers )
    
    - Recência em média: 33 dias
    
    - Compras em média: 144 compras
    
    - Receita em média: $2.610,89 dólares
    

### Cluster 03:

    - Número de customers: 349 (11% do customers )
    
    - Recência em média: 69 dias
    
    - Compras em média: 52 compras
    
    - Receita em média: $2.216,34 dólares
    

# 11.0 Deploy to production

In [96]:
#Plano
df92.dtypes

#TMUX

customer_id        int64
gross_revenue    float64
recency_days       int64
qtde_products      int64
frequency        float64
qtde_returns       int64
cluster            int64
dtype: object

## 11.1 Insert SQlite

In [102]:
import sqlite3
from sqlalchemy import create_engine

In [98]:
#create table
query_create_table_insiders= """
    CREATE TABLE insiders (
        customer_id      INTEGER,
        gross_revenue    REAL,
        recency_days     INTEGER,
        qtde_products    INTEGER,
        frequency        REAL,
        qtde_returns     INTEGER,
        cluster          INTEGER
               
)
"""

conn = sqlite3.connect('db_insiders.sqlite')
conn.execute(query_create_table_insiders)
conn.commit()
conn.close()

In [103]:
# insert data
conn = create_engine('sqlite:///db_insiders.sqlite')
df92.to_sql('insiders', con=conn, if_exists='append', index=False)

5695

In [104]:
#consulting database
query = """
    SELECT * FROM insiders
"""

df = pd.read_sql_query(query, conn)

In [105]:
df.head()

,customer_id,gross_revenue,recency_days,qtde_products,frequency,qtde_returns,cluster
0,17850,5391.21,372,297,17.000000,40,0
1,13047,3232.59,56,171,0.028302,35,0
2,12583,6705.38,2,232,0.040323,50,0
3,13748,948.25,95,28,0.017921,0,4
4,15100,876.00,333,3,0.073171,22,2


In [107]:
df.shape

(5695, 7)